<div style="text-align:center"><span style="color:blue; font-family:Times New Roman; font-size:3em;"> Data Preprocessing </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;padding-left:3em;padding-right:3em;"> The goal of this project is to predict which previous purchased products will be reordered in the latest order of the users in the test set (I call them as the test users in the following discussion). I divided the whole dataset into train and test data. For the test data, the feature variables were extracted from the prior transaction history and the general information of previous purchased products of the test users, which will be fed into a well-trained model to predict the reordered products in the latest order of the test users.  
<br \>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Suppose the order sequence of a user is ranked as 1, 2, 3, …., n-2, n-1, n,  where n is the latest order. For the train data, I utilized the transaction history before the (n-1)-th (in another word, the features are all for up to n-2 and the responses are for n-1 th order).The output of the train data would be whether the previous purchased products are reordered in the (n-1)-th order of the test users. The train data will be trained by a neural network in the next stage. 
 </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;padding-left:3em;padding-right:3em;">  For example, we have 22 orders of records for user_id = 15. Our train dataset features are everything upto 20th order and responses are whether those items upto 20th order are reordered or not in 21th order. (ie reponses, binary 0 ,1 is from 21th order). 
1) Train dataset: up to 20th order features and 21st order responses --> will split into validation within this train in the next modeling notebook.2) Similar logic for test dataset:  up to 21st order features and 22nd order responses 

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import copy

In [3]:
'''

path = "C:/Users/wanyi/Desktop/Research Design/Data/instacart/Original Data/"

aisles = pd.read_csv(path + 'aisles.csv')
departments = pd.read_csv(path +'departments.csv')
products = pd.read_csv(path +'products.csv')
prior = pd.read_csv(path +'order_products_prior.csv')
train = pd.read_csv(path +'order_products_train.csv')
orders = pd.read_csv(path +'orders.csv')

prior_train=pd.concat([prior, train],ignore_index=True)
prior_train=pd.merge(prior_train, orders, on='order_id', how='outer')
prior_train=pd.merge(prior_train, products, on='product_id', how='left')
prior_train=prior_train.sort_values(by=['user_id', 'order_number','product_id'])
prior_train.to_csv ('prior_train_final.csv', index=False)

prior_train = pd.read_csv('prior_train_final.csv')

prior_train.eval_set.value_counts()

'''



'\n\npath = "C:/Users/wanyi/Desktop/Research Design/Data/instacart/Original Data/"\n\naisles = pd.read_csv(path + \'aisles.csv\')\ndepartments = pd.read_csv(path +\'departments.csv\')\nproducts = pd.read_csv(path +\'products.csv\')\nprior = pd.read_csv(path +\'order_products_prior.csv\')\ntrain = pd.read_csv(path +\'order_products_train.csv\')\norders = pd.read_csv(path +\'orders.csv\')\n\nprior_train=pd.concat([prior, train],ignore_index=True)\nprior_train=pd.merge(prior_train, orders, on=\'order_id\', how=\'outer\')\nprior_train=pd.merge(prior_train, products, on=\'product_id\', how=\'left\')\nprior_train=prior_train.sort_values(by=[\'user_id\', \'order_number\',\'product_id\'])\nprior_train.to_csv (\'prior_train_final.csv\', index=False)\n\nprior_train = pd.read_csv(\'prior_train_final.csv\')\n\nprior_train.eval_set.value_counts()\n\n'

In [4]:
'''
#select 20k users 
insta = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Data/instacart/instacart_dataset.csv")
insta.rename(columns = {"db_food_id":"product_id"}, inplace =True)

prior_train = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Recommendation/Kaggle example/prior_train_james.csv")
insta = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Data/instacart/instacart_dataset.csv")
insta.rename(columns = {"db_food_id":"product_id"}, inplace =True)
sampled_user = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Recommendation/20kusers_new.csv")
sampled_user_id_list = sampled_user.user_id.tolist()

insta_col = insta[prior_train.columns.tolist()]
prior_train_james = insta_col[insta_col.user_id.isin(sampled_user_id_list)].reset_index(drop= True)
wanted_test_user_id = sorted(set(prior_train_james.user_id.tolist()))

test_old = prior_train[prior_train.eval_set == "test"]
test_old_df = test_old[test_old.user_id.isin(wanted_test_user_id)]

df = pd.concat([prior_train_james,test_old_df])
prior_train = copy.deepcopy(df)
prior_train.to_csv("prior_train_james_v3.csv")
'''

'\n#select 20k users \ninsta = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Data/instacart/instacart_dataset.csv")\ninsta.rename(columns = {"db_food_id":"product_id"}, inplace =True)\n\nprior_train = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Recommendation/Kaggle example/prior_train_james.csv")\ninsta = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Data/instacart/instacart_dataset.csv")\ninsta.rename(columns = {"db_food_id":"product_id"}, inplace =True)\nsampled_user = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Recommendation/20kusers_new.csv")\nsampled_user_id_list = sampled_user.user_id.tolist()\n\ninsta_col = insta[prior_train.columns.tolist()]\nprior_train_james = insta_col[insta_col.user_id.isin(sampled_user_id_list)].reset_index(drop= True)\nwanted_test_user_id = sorted(set(prior_train_james.user_id.tolist()))\n\ntest_old = prior_train[prior_train.eval_set == "test"]\ntest_old_df = test_old[test_old.user_id.isin(wanted_test_user_id)]\n\ndf = pd.con

In [5]:
OG_prior_train=pd.read_csv('prior_train_james_v3.csv') # prior_train_final

In [6]:
OG_prior_train = OG_prior_train[OG_prior_train.eval_set != "test"]

In [7]:
upp_bound = (OG_prior_train.groupby("user_id").agg({"order_number":"max"})-1).reset_index()
upp_bound[upp_bound.user_id == 11]

,user_id,order_number
0,11,6


In [8]:
prior_train = pd.merge(OG_prior_train,upp_bound, on ="user_id", how = "left")
prior_train.rename(columns = {"order_number_x":"order_number","order_number_y":"upper_bound"},inplace = True)

In [9]:
prior_train = prior_train[prior_train.order_number <= prior_train.upper_bound]

In [10]:
prior_train["product_id"] = prior_train.product_id.astype(int)

In [11]:
prior_train.head()

,Unnamed: 0,Unnamed: 0.1,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,upper_bound
0,0,84,170,53634,1.0,1.0,182389,train,1,0,13,14.0,asian chopped salad with dressing,123.0,4.0,6
1,1,85,170,53635,2.0,1.0,182389,train,1,0,13,14.0,xl emerald white seedless grapes,123.0,4.0,6
2,2,86,170,33482,3.0,1.0,182389,train,1,0,13,14.0,bag of organic bananas,24.0,4.0,6
3,3,87,170,53636,4.0,1.0,182389,train,1,0,13,14.0,bbq ranch chopped salad,123.0,4.0,6
4,4,88,170,11216,5.0,1.0,182389,train,1,0,13,14.0,100 percent whole wheat bread,112.0,3.0,6


In [25]:
avg_num_item_per_basket = pd.DataFrame(prior_train.groupby(["user_id","order_id"]).agg({"product_id": "count"})).reset_index()
avg_num_item_per_basket.rename(columns = {"product_id":"order_size"}, inplace =True)

In [31]:
avg_num_item_per_basket.order_size.mean()

9.627922418754071

In [41]:
food_db = pd.read_csv("C:/Users/wanyi/Desktop/Research Design/Data/Nutrition data/Nutrition Data Consolidated (James)_Sept_22_with_type_scores_price.csv")
food_db = food_db[food_db.Source == "Nutritionix"][["food_id","SAIN_LIM_Point"]]
food_db.rename(columns = {"food_id":"product_id"},inplace = True)

<div style="text-align:left"><span style="color:blue; font-family:Times New Roman; font-size:1.8em;">  
The train data preparation  </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">The train data involves the test users and the non-test users. The users are selected as the test users via eval_set='test' from the preprocessed table (prior_train). Once the test users are chosen, all the purchased items before the (n-1)-th order for a given test user are selected. The feature input of a given purchased product for a given test user is built upon the interaction between the item and the user as well as 
the general information of the item and the user. The output of a given purchased product for a given test user is whether the product is purchased in the (n-1)-th order of the user. I call the output as ‘reordered’.  
 </span></div>

In [15]:
test_users=sorted(set(prior_train[prior_train.eval_set=='prior'].user_id.values))
X_test_users=pd.DataFrame(test_users, columns=['user_id'])
prior_train_test_users=pd.merge(X_test_users, prior_train, on='user_id', how='left')
total_orders_user=prior_train_test_users[prior_train_test_users.eval_set=='prior'].groupby('user_id')['order_number'].max()
prior_train_test_users['total_orders_user']=prior_train_test_users.user_id.map(total_orders_user)
X_train_test_users=prior_train_test_users[prior_train_test_users.order_number < prior_train_test_users.total_orders_user] #select upto 21th order for user 15
## build 'user_id','product_id'
#print 'build user_id,product_id'
userid_productid=X_train_test_users.groupby(['user_id','product_id'])['order_number'].size().reset_index()
X_test_users=pd.merge(X_test_users,userid_productid[['user_id','product_id']], on='user_id', how='left')
#### important notes on what is "reordered" in X and X_test in the later stage!
Y_train_test_users=prior_train_test_users[(prior_train_test_users.order_number ==\
                                          prior_train_test_users.total_orders_user)&\
                                          (prior_train_test_users.reordered==1)][['user_id','product_id', 'reordered']] 
#important!!!!!!!!!!!!!!
#see in the order 22 which ones were reordered. so these items reorded are denoted as 1. 
#Nerual network will learn internal relatoinship on how next basket will include those reorded again. 
#The ones are new in the next order are not considered here.
X_test_users=pd.merge(X_test_users, Y_train_test_users, on=['user_id','product_id'], how='left').fillna(0)


In [16]:
Y_train_test_users[Y_train_test_users.user_id ==15] #in order 22, only 40016 and 36075 are reordered before 

,user_id,product_id,reordered


In [17]:
X_test_users[X_test_users.user_id == 15]

,user_id,product_id,reordered


<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">The features for a given purchased product of a given user are as follows:
<br \>
total_orders_user&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp; Total orders for the user. 
<br \>
unique_items_user&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Total unique products for the user.
<br \>
total_items_user&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Total products for the user.
<br \>
average_items_per_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Average items per order for the user.
<br \>
average_days_per_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Average days between each order for the user.
<br \>
appear_rate&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Appear rate for the product in the user baskets.
<br \>
reorder_num&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Total reorder times for the product purchased by the user.
<br \>
add_to_cart_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Mean order of adding the product to the cart by the user.
<br \>
total_order_minus_last_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Order interval between the last order of the product and the (n-1)-th order of the test user (the n-th order of the non-test user). 
<br \>
last_order_ratio&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The ratio of the last order of the product to the (n-1)-th order of the test user (the n-th order of the non-test user). 
<br \>
aisle_id&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Aisle ID of the product.
<br \>
department_id&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Department ID of the product.
<br \>
user_num_product&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The number of the users who buy the product among all users.
<br \>
product_total_orders&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The number of the orders who contain the product in all users.
<br \>
product_total_reorders&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The number of the re-orders who contain the product in all users.
<br \>
reorder_ratio&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;The ratio of product_total_reorders to product_total_orders.
<br \>
last_kst_order&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;k is 1, 2, ..., 19, 20. Whether the product is purchased in the (n-1-k)-th order of the test user (the (n-k)-th order of the non-test user). It is a Boolean value.
<br \>
last_five_orders_sum&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Sum of last_1st_order, last_2st_order, .., and last_5st_order
<br \>
last_10_orders_sum&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Sum of last_1st_order, last_2st_order, .., and last_10st_order
<br \>
last_15_orders_sum&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Sum of last_1st_order, last_2st_order, .., and last_15st_order
<br \>
last_20_orders_sum&nbsp;&nbsp;&nbsp;&nbsp;:&nbsp;Sum of last_1st_order, last_2st_order, .., and last_20st_order
<br \>
I created a function (transform_data) to transform the data in the preprocessed table (prior_train) to the feature variables for the test users.  
 </span></div>

In [18]:
def transform_data(df_X_train,X_train, status):
    #print(X_train.shape)
    #### total orders for an individual user
    print('total orders for an individual user')
    total_orders_user=df_X_train.groupby('user_id')['order_number'].max()
    X_train['total_orders_user']=X_train.user_id.map(total_orders_user)
    #### total unique products for an individual user
    print('total unique products for an individual user')
    def cal1(x):
        return x.nunique()
    unique_items_user=df_X_train.groupby('user_id')['product_id'].apply(cal1)
    X_train['unique_items_user']=X_train.user_id.map(unique_items_user)
    #### total items for an individual user
    print('total items for an individual user')
    total_items_user=df_X_train.groupby('user_id')['product_id'].size()
    X_train['total_items_user']=X_train.user_id.map(total_items_user)
    #### average items per order for an individual user
    X_train['average_items_per_order']=X_train.total_items_user/X_train.total_orders_user
    #### average days between each order for an individual user
    print('average days between each order for an individual user')
    days_per_order=df_X_train.groupby(['user_id','order_number'])['days_since_prior_order'].agg('mean')
    average_days_per_order=days_per_order.dropna().reset_index()
    average_days_per_order=average_days_per_order.groupby(['user_id'])['days_since_prior_order'].mean()
    X_train['average_days_per_order']=X_train.user_id.map(average_days_per_order)
    #### appear rate for this product in the user baskets
    print('appear rate for this product in the user baskets')
    appear_num=df_X_train.groupby(['user_id','product_id'])['order_number'].agg('count').reset_index()
    appear_num=appear_num.rename(columns={'order_number':'appear_num'})
    X_train=pd.merge(X_train, appear_num, on=['user_id','product_id'], how='left')
    X_train['appear_rate']=X_train.appear_num/X_train.total_orders_user
    X_train.drop('appear_num', axis=1, inplace=True)
    #### total reorder number for this product for this user
    print('total reorder number for this product for this user')
    reorder_num=df_X_train.groupby(['user_id','product_id'])['reordered'].agg([('reorder_num', 'sum')]).reset_index()
    X_train=pd.merge(X_train, reorder_num, on=['user_id','product_id'], how='left')
    #### mean values of add to cart order for this product for this user
    print('mean values of add to cart order for this product for this user')
    add_to_cart_order_mean=df_X_train.groupby(['user_id','product_id'])['add_to_cart_order'].agg('mean').reset_index()
    X_train=pd.merge(X_train, add_to_cart_order_mean, on=['user_id','product_id'], how='left')
    #### order interval for this product for this user
    print('order interval for this product for this user')
    def cal2(x):
        if x.nunique()==1:
            return x.max()
        else:
            return x.unique()[-2]
    
    df_X_train["last_order"] =  df_X_train["order_number"].tolist()
    df_X_train["last_second_order"] =  df_X_train["order_number"].tolist()
    
    #order_interval_product=df_X_train.groupby(['user_id','product_id'])['order_number'].agg({'last_order':max,'last_second_order':cal2}).reset_index()
    order_interval_product=df_X_train.groupby(['user_id','product_id']).agg({'last_order':max,'last_second_order':cal2}).reset_index()
    
    
    X_train=pd.merge(X_train, order_interval_product, on=['user_id','product_id'], how='left')
    X_train['total_order_minus_last_order']=X_train['total_orders_user']-X_train['last_order']
    #### last order of this product / total orders
    X_train['last_order_ratio']=X_train['last_order'].values / np.array(X_train['total_orders_user'].values, dtype='float')
    X_train.drop(['last_second_order','last_order'], axis=1, inplace=True)
    #### 'aisle_id','department_id' for a product
    print('aisle_id,department_id for a product')
    aisle_dep=df_X_train.groupby(['user_id','product_id'])['aisle_id','department_id'].agg('mean').reset_index()
    X_train=pd.merge(X_train, aisle_dep, on=['user_id','product_id'], how='left')
    #### the user number who bought this product
    print('the user number who bought this product')
    def cal3(x):
        return x.nunique()
    user_num_product=df_X_train.groupby('product_id')['user_id'].apply(cal3)
    X_train['user_num_product']=X_train.product_id.map(user_num_product)
    #### orders which have this product
    print('orders which have this product')
    product_total_orders=df_X_train.groupby('product_id')['user_id'].size()
    X_train['product_total_orders']=X_train.product_id.map(product_total_orders)
    #### re-orders which have this product
    print('re-orders which have this product')
    product_total_reorders=df_X_train[df_X_train.reordered==1].groupby('product_id')['user_id'].size()
    X_train['product_total_reorders']=X_train.product_id.map(product_total_reorders)
    X_train['reorder_ratio']=X_train['product_total_reorders']/X_train['product_total_orders']
    #### order sequence for this product (one hot encoder) 
    print('order sequence (one hot encoder) for this product (last 20 orders for this user)')
    df_X_train['total_order_minus_order_number']=df_X_train.total_orders_user - df_X_train.order_number
    for i in tqdm(range(1,21)):
        order_sequence_binary=df_X_train[df_X_train.\
                                         total_order_minus_order_number==i][['user_id','product_id','total_order_minus_order_number']]
        order_sequence_binary['total_order_minus_order_number']=order_sequence_binary.total_order_minus_order_number / i
        order_sequence_binary=order_sequence_binary.rename(columns={'total_order_minus_order_number':'last_{}st_order'.format(i)})
        X_train=pd.merge(X_train, order_sequence_binary, on=['user_id','product_id'], how='left').fillna(0)
    if status == 'train':
        X_train['last_five_orders_sum']=X_train.iloc[:,21:26].sum(1)
        X_train['last_10_orders_sum']=X_train.iloc[:,21:31].sum(1)
        X_train['last_15_orders_sum']=X_train.iloc[:,21:36].sum(1)
        X_train['last_20_orders_sum']=X_train.iloc[:,21:41].sum(1) 
    else: 
        X_train['last_five_orders_sum']=X_train.iloc[:,20:25].sum(1)
        X_train['last_10_orders_sum']=X_train.iloc[:,20:30].sum(1)
        X_train['last_15_orders_sum']=X_train.iloc[:,20:35].sum(1)
        X_train['last_20_orders_sum']=X_train.iloc[:,20:40].sum(1) 
    print(X_train.shape)
    return(X_train)

In [19]:
X_test_users=transform_data(X_train_test_users,X_test_users,'train')

total orders for an individual user
total unique products for an individual user
total items for an individual user
average days between each order for an individual user
appear rate for this product in the user baskets
total reorder number for this product for this user
mean values of add to cart order for this product for this user
order interval for this product for this user
aisle_id,department_id for a product
the user number who bought this product
orders which have this product
re-orders which have this product


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

order sequence (one hot encoder) for this product (last 20 orders for this user)


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:29<00:00,  1.47s/it]


(1149112, 43)


## Fixing inbalance dataset (did not perform)

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">The train data only involving the test users will be suffered to the imbalanced issue.  Almost 90% of the data belongs to the class 0 (reordered=0) can cause 90% accuracy immediately while being trained by a classification model.  This is because the learning algorithm of a model tends to be biased towards the majority class and classifies every example as the majority class.
<br \>
So what is my strategy  when the data is imbalanced?
<br \>
1)	Oversample the minority class.
<br \>
2)	Undersample the majority class. 
<br \>
3)	Adjust the class weight in the learning algorithm. (discussed in the next stage)
<br \>
In order to increase the sample number of the minority class, I included the non-test users into the train data.  The non-test users are selected via eval_set='train' from the preprocessed table (prior_train). I only considered the purchased products that are reordered in the n-th order of the non-test users. Therefore, the ‘reordered’ output of these products are all ‘True’ (1). The ‘transform_data’ function transforms the information of these selected purchased products to the feature variables for the non-test users. 
 </span></div>

In [20]:
'''
userid_productid=prior_train[(prior_train.eval_set=='train')&(prior_train.reordered==1)][['user_id','product_id','reordered']].reset_index()
X_train_users=userid_productid[['user_id','product_id','reordered']]
train_users=pd.DataFrame(X_train_users.user_id.unique(), columns=['user_id'])
prior_train_selected=pd.merge(train_users, prior_train[prior_train.eval_set=='prior'], on='user_id', how='left')
total_orders_user=prior_train.groupby('user_id')['order_number'].max()
prior_train_selected['total_orders_user']=prior_train_selected.user_id.map(total_orders_user)
'''

"\nuserid_productid=prior_train[(prior_train.eval_set=='train')&(prior_train.reordered==1)][['user_id','product_id','reordered']].reset_index()\nX_train_users=userid_productid[['user_id','product_id','reordered']]\ntrain_users=pd.DataFrame(X_train_users.user_id.unique(), columns=['user_id'])\nprior_train_selected=pd.merge(train_users, prior_train[prior_train.eval_set=='prior'], on='user_id', how='left')\ntotal_orders_user=prior_train.groupby('user_id')['order_number'].max()\nprior_train_selected['total_orders_user']=prior_train_selected.user_id.map(total_orders_user)\n"

In [21]:
'''
X_train_users=transform_data(prior_train_selected,X_train_users,'train')
'''

"\nX_train_users=transform_data(prior_train_selected,X_train_users,'train')\n"

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">
In order to undersample the majority class (‘reordered’=0) of the train data, I deleted most of the samples that the products don't appear within last 20 orders, and remained only one for each test user. This is because if a product is not purchased within the last 20 orders of a given user, the chance of getting reordered (‘reordered’=1) in the latest order is tiny. This method substantially decreases the sample number of the class 0.

 </span></div>

In [22]:
'''
reordered_0=X_test_users[(X_test_users.last_20_orders_sum==0)&(X_test_users.reordered==0)]
reordered_1=X_test_users[(X_test_users.last_20_orders_sum==0)&(X_test_users.reordered==1)]
X_test_users=X_test_users[X_test_users.last_20_orders_sum!=0]
reordered_0_seleted=reordered_0.groupby(['user_id'])['product_id'].min().reset_index()
reordered_0=pd.merge(reordered_0_seleted, reordered_0, on=['user_id', 'product_id'], how='left')
X_test_users=pd.concat([X_test_users, reordered_0, reordered_1],ignore_index=True)
'''

"\nreordered_0=X_test_users[(X_test_users.last_20_orders_sum==0)&(X_test_users.reordered==0)]\nreordered_1=X_test_users[(X_test_users.last_20_orders_sum==0)&(X_test_users.reordered==1)]\nX_test_users=X_test_users[X_test_users.last_20_orders_sum!=0]\nreordered_0_seleted=reordered_0.groupby(['user_id'])['product_id'].min().reset_index()\nreordered_0=pd.merge(reordered_0_seleted, reordered_0, on=['user_id', 'product_id'], how='left')\nX_test_users=pd.concat([X_test_users, reordered_0, reordered_1],ignore_index=True)\n"

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">
Finally, the feature variables and output variables from the test user and non-test users are combined to create the train data and saved in ‘X.csv’ for the next stage of data modeling. 
The percentages of the class 0 (‘reordered’=0) and class 1 (‘reordered’=1) in the train data are 72.6% and 27.4 %.  
 </span></div>

In [78]:
'''
X=pd.concat([X_test_users, X_train_users],ignore_index=True)
X.to_csv('X_v3.csv')
'''
X_v3 = pd.read_csv('X_v3.csv')
X = pd.merge(X_v3, food_db, on="product_id", how ="left")
X.to_csv('X_v4.csv')
X = pd.read_csv('X_v4.csv')

<div style="text-align:left"><span style="color:blue; font-family:Times New Roman; font-size:1.8em;">  
The test data preparation  </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;">For the test data, the feature variables are extracted from the transaction history before the n-th order of the test users. The output of whether a purchased product is reordered in the n-th order of a given test user will be predicted by a trained model. The features of the test data are created by the ‘transform_data’ function and saved as ‘X_test.csv’. 
 </span></div>

In [26]:
test_users=sorted(set(prior_train[prior_train.eval_set=='prior'].user_id.values))
X_test_users=pd.DataFrame(test_users, columns=['user_id'])
prior_train_test_users=pd.merge(X_test_users, prior_train, on='user_id', how='left')
total_orders_user=prior_train_test_users.groupby('user_id')['order_number'].max()
prior_train_test_users['total_orders_user']=prior_train_test_users.user_id.map(total_orders_user)
prior_train_test_users=prior_train_test_users[prior_train_test_users.eval_set=='prior']
## build 'user_id','product_id'
#print 'build user_id,product_id'
userid_productid=prior_train_test_users.groupby(['user_id','product_id'])['order_number'].size().reset_index()
X_test_users=pd.merge(X_test_users,userid_productid[['user_id','product_id']], on='user_id', how='left')

In [27]:
X_test=transform_data(prior_train_test_users,X_test_users,'test')

total orders for an individual user
total unique products for an individual user
total items for an individual user
average days between each order for an individual user
appear rate for this product in the user baskets
total reorder number for this product for this user
mean values of add to cart order for this product for this user
order interval for this product for this user
aisle_id,department_id for a product
the user number who bought this product
orders which have this product
re-orders which have this product


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

order sequence (one hot encoder) for this product (last 20 orders for this user)


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:32<00:00,  1.64s/it]


(1178576, 42)


In [28]:
X_test.shape

(1178576, 42)

In [29]:
# this X_test is the state (snapshot) that as of order 22 for user 15. It includes all the food that user have seen in the past up to order 22.

X_test.to_csv('X_test_v3.csv')

In [80]:
X_test_v3 = pd.read_csv('X_test_v3.csv')
X_test_v4 = pd.merge(X_test_v3,food_db, on="product_id", how ="left")
X_test_v4.to_csv('X_test_v4.csv')
X_test = pd.read_csv('X_test_v4.csv')

In [81]:
len(sorted(set(X_test_v4.user_id.values)))

20388

In [82]:
len(sorted(set(X.user_id.values)))

20388

# James Check

In [34]:
prior_train[prior_train.user_id == 11].tail(20)

,Unnamed: 0,Unnamed: 0.1,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,upper_bound
2326849,2326849,22882690,2411567,37851.0,6.0,0.0,11,prior,5,0,11,NaN,the inchworld's best inch veggie burger,42.0,1.0,6
2326850,2326850,22882691,2411567,40603.0,7.0,0.0,11,prior,5,0,11,NaN,slow cured pepper plus garlic salame molisana,96.0,20.0,6
2326851,2326851,22882692,2411567,40761.0,8.0,0.0,11,prior,5,0,11,NaN,calabrese uncured italian salami with pepper a...,96.0,20.0,6
2326852,2326852,22882693,2411567,9466.0,9.0,0.0,11,prior,5,0,11,NaN,hot dog buns,43.0,3.0,6
2326853,2326853,22882694,2411567,42094.0,10.0,0.0,11,prior,5,0,11,NaN,italian extra virgin olive oil,19.0,13.0,6
2326854,2326854,22882695,2411567,33366.0,11.0,0.0,11,prior,5,0,11,NaN,large grapefruit,24.0,4.0,6
2326855,2326855,22882696,2411567,4238.0,12.0,0.0,11,prior,5,0,11,NaN,beer brats sausage,14.0,20.0,6
2716675,2716675,26732735,2840752,55118.0,1.0,1.0,11,prior,6,5,10,14.0,lemonade,31.0,7.0,6
2716676,2716676,26732736,2840752,33947.0,2.0,1.0,11,prior,6,5,10,14.0,pure sparkling water,115.0,7.0,6
2716677,2716677,26732737,2840752,33982.0,3.0,1.0,11,prior,6,5,10,14.0,tonic water,115.0,7.0,6


In [75]:
X[X.user_id == 11]

,Unnamed: 0,Unnamed: 0.1,user_id,product_id,reordered,total_orders_user,unique_items_user,total_items_user,average_items_per_order,average_days_per_order,...,last_16st_order,last_17st_order,last_18st_order,last_19st_order,last_20st_order,last_five_orders_sum,last_10_orders_sum,last_15_orders_sum,last_20_orders_sum,SAIN_LIM_Point
0,0,0,11,4238.0,1.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1
1,1,1,11,4310.0,1.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,6
2,2,2,11,4880.0,0.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,6
3,3,3,11,5036.0,0.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,5
4,4,4,11,6781.0,0.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,5
5,5,5,11,6783.0,0.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,5
6,6,6,11,7118.0,0.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,6
7,7,7,11,7159.0,0.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
8,8,8,11,8820.0,0.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
9,9,9,11,9466.0,0.0,5,46,60,12.0,19.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5


In [71]:
X_test[X_test.user_id == 11]

,Unnamed: 0,Unnamed: 0.1,user_id,product_id,total_orders_user,unique_items_user,total_items_user,average_items_per_order,average_days_per_order,appear_rate,...,last_16st_order,last_17st_order,last_18st_order,last_19st_order,last_20st_order,last_five_orders_sum,last_10_orders_sum,last_15_orders_sum,last_20_orders_sum,SAIN_LIM_Point
0,0,0,11,4238.0,6,48,73,12.166667,18.6,0.500000,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,1
1,1,1,11,4310.0,6,48,73,12.166667,18.6,0.500000,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,6
2,2,2,11,4880.0,6,48,73,12.166667,18.6,0.166667,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,6
3,3,3,11,5036.0,6,48,73,12.166667,18.6,0.166667,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,5
4,4,4,11,6781.0,6,48,73,12.166667,18.6,0.166667,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,5
5,5,5,11,6783.0,6,48,73,12.166667,18.6,0.166667,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,5
6,6,6,11,7118.0,6,48,73,12.166667,18.6,0.333333,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,6
7,7,7,11,7159.0,6,48,73,12.166667,18.6,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
8,8,8,11,8820.0,6,48,73,12.166667,18.6,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
9,9,9,11,9466.0,6,48,73,12.166667,18.6,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
